#Desicion Tree Classifier

###Objetivo
Predecir a traves de regresion logistica la hipotesis
Hipótesis de clasificación:
"*¿Es posible predecir si un jugador sobrevivirá en una ronda?*"

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

##Carga de los datos

In [ ]:
from google.colab import files
import pandas as pd # Importing the pandas library

uploaded = files.upload()
for fn in uploaded.keys():
    name=fn
df = pd.read_csv(name, sep=";") # Now pd is defined and can be used


Saving Anexo ET_demo_round_traces_2022 (1) (1).csv to Anexo ET_demo_round_traces_2022 (1) (1).csv


##Limpieza de datos



In [ ]:
df.head()

,Unnamed: 0,Map,Team,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,...,RoundKills,RoundAssists,RoundHeadshots,RoundFlankKills,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots
0,600,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,750,4400,0,0,0,0
1,601,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,800,4400,0,0,0,0
2,602,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,1000,4400,0,0,0,0
3,603,de_inferno,Terrorist,1,4,1,False,True,False,False,...,0,0,0,0,850,4400,0,0,0,0
4,604,de_inferno,Terrorist,1,4,1,False,True,True,False,...,0,0,0,0,1000,4400,0,0,0,0


In [ ]:
df2 = df.copy()

In [ ]:
df2.info()
print("\nSample of 'TimeAlive' values:")
print(df2['TimeAlive'].head())
print("\nSample of 'TravelledDistance' values:")
print(df2['TravelledDistance'].head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79157 entries, 0 to 79156
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   79157 non-null  int64  
 1   Map                          79157 non-null  object 
 2   Team                         79155 non-null  object 
 3   InternalTeamId               79157 non-null  int64  
 4   MatchId                      79157 non-null  int64  
 5   RoundId                      79157 non-null  int64  
 6   RoundWinner                  79157 non-null  object 
 7   MatchWinner                  79156 non-null  object 
 8   Survived                     79157 non-null  bool   
 9   AbnormalMatch                79157 non-null  bool   
 10  TimeAlive                    79157 non-null  object 
 11  TravelledDistance            79157 non-null  object 
 12  RLethalGrenadesThrown        79157 non-null  int64  
 13  RNonLethalGrenad

In [ ]:
#vemos si hay datos faltantes
df2.columns[df.isnull().any()]

Index(['Team', 'MatchWinner'], dtype='object')

In [ ]:
#como son pocos, eliminamos las filas con valores nulos
# Eliminar todas las filas que contengan al menos un valor nulo
df2 = df2.dropna()

# Confirmar cuántas filas quedaron
print(f"Filas restantes después de eliminar nulos: {len(df2)}")


Filas restantes después de eliminar nulos: 79154


In [ ]:
# Forzar conversión de columnas a tipo booleano primero, luego a int (0 o 1)
cols = ['RoundWinner', 'MatchWinner', 'Survived', 'AbnormalMatch']

for col in cols:
    df2[col] = df2[col].astype('bool').astype('int')

In [ ]:
# Aplicar One-Hot Encoding a la columna 'Team'
df2 = pd.get_dummies(df2, columns=['Team'], drop_first=True)

# Verificar qué columnas se crearon
team_cols = [col for col in df.columns if col.startswith('Team_')]
print("Columnas creadas para Team:", team_cols)


Columnas creadas para Team: []


In [ ]:
df2['Map'].unique()

array(['de_inferno', 'de_nuke', 'de_mirage', 'de_dust2'], dtype=object)

In [ ]:
# Aplicar One-Hot Encoding a la columna 'Map'
df2 = pd.get_dummies(df2, columns=['Map'], drop_first=True)

# Verifica los nuevos nombres de columnas
print(df2.columns)

Index(['Unnamed: 0', 'InternalTeamId', 'MatchId', 'RoundId', 'RoundWinner',
       'MatchWinner', 'Survived', 'AbnormalMatch', 'TimeAlive',
       'TravelledDistance', 'RLethalGrenadesThrown',
       'RNonLethalGrenadesThrown', 'PrimaryAssaultRifle', 'PrimarySniperRifle',
       'PrimaryHeavy', 'PrimarySMG', 'PrimaryPistol', 'FirstKillTime',
       'RoundKills', 'RoundAssists', 'RoundHeadshots', 'RoundFlankKills',
       'RoundStartingEquipmentValue', 'TeamStartingEquipmentValue',
       'MatchKills', 'MatchFlankKills', 'MatchAssists', 'MatchHeadshots',
       'Team_Terrorist', 'Map_de_inferno', 'Map_de_mirage', 'Map_de_nuke'],
      dtype='object')


In [ ]:
# Selecciona todas las columnas que son booleanas
bool_cols = ['Team_Terrorist', 'Map_de_inferno', 'Map_de_mirage', 'Map_de_nuke']

# Convierte True/False a 1/0
df2[bool_cols] = df2[bool_cols].astype(int)

# Verificación
print(df2[bool_cols].head())


   Team_Terrorist  Map_de_inferno  Map_de_mirage  Map_de_nuke
0               1               1              0            0
1               1               1              0            0
2               1               1              0            0
3               1               1              0            0
4               1               1              0            0


In [ ]:
df2 = df2.drop(['FirstKillTime'], axis=1)

In [ ]:
df2 = df2.drop(['TimeAlive', 'TravelledDistance'], axis=1)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79154 entries, 0 to 79156
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   79154 non-null  int64  
 1   InternalTeamId               79154 non-null  int64  
 2   MatchId                      79154 non-null  int64  
 3   RoundId                      79154 non-null  int64  
 4   RoundWinner                  79154 non-null  int64  
 5   MatchWinner                  79154 non-null  int64  
 6   Survived                     79154 non-null  int64  
 7   AbnormalMatch                79154 non-null  int64  
 8   RLethalGrenadesThrown        79154 non-null  int64  
 9   RNonLethalGrenadesThrown     79154 non-null  int64  
 10  PrimaryAssaultRifle          79154 non-null  float64
 11  PrimarySniperRifle           79154 non-null  float64
 12  PrimaryHeavy                 79154 non-null  float64
 13  PrimarySMG           

In [ ]:
df2.head()

,Unnamed: 0,InternalTeamId,MatchId,RoundId,RoundWinner,MatchWinner,Survived,AbnormalMatch,RLethalGrenadesThrown,RNonLethalGrenadesThrown,...,RoundStartingEquipmentValue,TeamStartingEquipmentValue,MatchKills,MatchFlankKills,MatchAssists,MatchHeadshots,Team_Terrorist,Map_de_inferno,Map_de_mirage,Map_de_nuke
0,600,1,4,1,1,1,0,0,0,4,...,750,4400,0,0,0,0,1,1,0,0
1,601,1,4,1,1,1,0,0,0,0,...,800,4400,0,0,0,0,1,1,0,0
2,602,1,4,1,1,1,0,0,0,0,...,1000,4400,0,0,0,0,1,1,0,0
3,603,1,4,1,1,1,0,0,0,0,...,850,4400,0,0,0,0,1,1,0,0
4,604,1,4,1,1,1,1,0,1,1,...,1000,4400,0,0,0,0,1,1,0,0


##Entrenar modelo


In [ ]:
X = df2.drop('Survived', axis=1)
y = df2['Survived']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:


# Crear el modelo con una semilla para reproducibilidad
clf = DecisionTreeClassifier(random_state=42)


In [ ]:
# Entrenar el modelo con los datos de entrenamiento
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

##Prediccion y evaluacion

In [ ]:
# Predecir con los datos de prueba
y_pred = clf.predict(X_test)


In [ ]:


# Métricas principales
print(" Accuracy:", accuracy_score(y_test, y_pred))
print("\n Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("\n Matriz de Confusión:")
print(cm)

# Desglosar FP, FN, TP, TN si deseas:
tn, fp, fn, tp = cm.ravel()
print(f"\n Verdaderos Positivos (TP): {tp}")
print(f" Falsos Positivos (FP): {fp}")
print(f" Falsos Negativos (FN): {fn}")
print(f" Verdaderos Negativos (TN): {tn}")


 Accuracy: 0.6304718590108016

 Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.69      0.69      9473
           1       0.54      0.54      0.54      6358

    accuracy                           0.63     15831
   macro avg       0.62      0.62      0.62     15831
weighted avg       0.63      0.63      0.63     15831


 Matriz de Confusión:
[[6542 2931]
 [2919 3439]]

 Verdaderos Positivos (TP): 3439
 Falsos Positivos (FP): 2931
 Falsos Negativos (FN): 2919
 Verdaderos Negativos (TN): 6542
